# MA10276 Programming coursework - Semester 2 supplementary

**Before making any changes to this notebook, save it as `coursework_USERNAME.ipynb` where `USERNAME` is your username. You need to save this new file in the `MA10276_workspace` directory.**


**DO NOT create any extra cells in this notebook** with the exception of additional cells under the Additional Examples section at the bottom of this file.

* Your code should be put in the relevant cell. 

* Any packages that you need to import should be put in Task 1 cell

* Code for Task 1.a, 1.b, 1.c should appear in the Task 1 cell. Running this cell **should not** produce any printed output.

* Code for Task 2.a, 2.b should appear in the Task 2 cell. Running this cell **should not** produce any printed output.

* The examples provided in Coursework should be put in the relevant cells, e.g. Task 1.a Example cell, Task 1.b Example cell, etc. In particular the examples will produce printed output and **should not** be put in the main cells for Task 1 and Task 2. 

* You should **NOT** add any additional tests, or prints to Task 1 cell, Task 2 cell, or Example cells described above. 

* Any additional tests that you ran for checking correctness of your code should be put under the **Additional Examples** section. Note that this is the only section under which you are allowed to add new cells. Note that all cells under **Additional Examples** section will be ignored and **will NOT be assessed**.

#### Enter your full name in the following cell:

Thomas Henderson

## Task 1

### Task 1 Code
Copy the skeletal code for Task 1 in the cell below and complete the implementation of Task 1.a-c as instructed. 

In [3]:
import numpy as np
import copy
import math

# Task 1.a
def clean(string):
    '''Returns the string inputted but removing the non-alphabetical characters.
    Input: String
    Output: Same string but consisting only of alphabetical characters'''
    new_str = ''
    for character in string:
        if character.isalpha() == True:
            new_str = new_str + character
    string = new_str
    return string


# Task 1.b
def matrix_to_str(A):
    ''' Converts a matrix into a string representation of it involving a header and then the rest of the data
    Input: A matrix
    Output: A string representing the inputted matrix'''
    string = '|'
    no_rows = A.shape[0]
    no_cols = A.shape[1]
    # First row 
    for j in range(no_cols):
        string = string + ' ' + str(A[0,j]) + ' |'
    string = string + '\n|'
    # Now the boundary
    for j in range(no_cols):
        string = string + '---|'
    string = string + '\n|'
    # Now for the intermediate columns
    for i in range(1,no_rows - 1):
        for j in range(no_cols):
            string = string + ' ' + str(A[i,j]) + ' |'
        string = string + '\n|'
    # Now do the final column
    for j in range(no_cols):
        string = string + ' ' + str(A[no_rows-1,j]) + ' |'
    return string


class TranspositionCipher(object):

    # Do not modify this method
    def __repr__(self):
        return str(self)

    # Task 1.a
    def __init__(self, rawkey='Unknown', null='Z'):
        '''Creates an instance of the class TranspositionCipher
        Input: The word for the key and what the null character is
        Output: An object of class Transposition Cipher '''
        self.rawkey = rawkey
        self.null = null
        # Convert the rawkey into the appropiate format for the cipher
        with_all_instances = clean(rawkey).lower()
        cipher = ''
        for character in with_all_instances:
            if character not in cipher:
                cipher = cipher + character
        self.key = cipher
        sorted_cipher = sorted(cipher)
        orders = [] # Will store the alphabetical order of each character in the cipher
        for character in cipher:
            order = sorted_cipher.index(character)
            orders.append(order)
        self.order = orders
        self.text = []
        
    # Task 1.a
    def __str__(self):
        '''Represents the TranspositionCipher object as a string
        Input: TranspositionCipher object
        Ouput: String containing information about the object'''
        return self.rawkey + ' { ' + self.key + ' : ' + str(self.order) + ' : ' + self.null + ' }'

    
    # Task 1.b
    def encode_message(self, message):
        ''' Encodes the string stored in the parameter message with the key from the object
        Input: The cipher object to be used and the message to be encoded
        Output: The encoded string, a matrix representing the rawkey and original message and a matrix
        representing the cipher version of the rawkey and the encoded message'''
        n = len(self.key)
        no_rows = int(math.ceil(len(message) / n)) + 1 # The plus one is for the key, ceil rounds up to nearest integer
        key = self.key
        M = np.full((no_rows,n), ' ', dtype=np.unicode_) # Creates empty matrix with number of storage slots we need
        # Fill first row with the characters from the key
        for i in range(n):
            M[0,i] = key[i]
        # Now fill as much of the rest of the matrix as possible with the message to be encoded
        counter = 0
        next_spot = [1,0] # This variable and counter will help us move through the matrix
        while counter < len(message): 
            M[next_spot[0],next_spot[1]] = message[counter]
            if next_spot[1] == n - 1: # Goes to a new row
                next_spot[1] = 0
                temp = next_spot[0]
                next_spot[0] = temp + 1
            else:
                temp = next_spot[1]
                next_spot[1] = temp + 1
            counter += 1
        # Now fill up the remaining spots in the matrix with the null values
        while next_spot[1] != 0:
            M[next_spot[0],next_spot[1]] = self.null
            if next_spot[1] == n - 1:
                next_spot[1] = 0
            else:
                temp = next_spot[1]
                next_spot[1] = temp + 1
        # After this Matrix M containing unencoded data is now completed. Let's convert it into Matrix C
        order = self.order
        C = np.full((no_rows,n), ' ', dtype=np.unicode_)
        # First column in similar fashion as M
        for i in range(n):
            C[0,order[i]] = key[i]
        # Now let's use the columns of M and paste them accordingly into C
        for j in range(n):
            for i in range(1,no_rows):
                C[i,order[j]] = M[i,j]
        # That is all the columns filled in as M is already a full matrix, so the null values are transferred over.
        # Finally let's convert this into our message reading down the columns
        message = ''
        for j in range(n):
            for i in range(1,no_rows):
                message = message + C[i,j]
        return message, M, C

    # Task 1.b
    def decode_message(self, encodedmessage):
        ''' Decodes the string stored in the parameter encodedmessage with the key from the object
        Input: The cipher object to be used and the message to be decoded
        Output: The decoded string, a matrix representing the cipher and encoded message and a matrix
        representing the rawkey and the decoded message'''
        # First we need to represent the encoded message in a matrix in much the same way
        # as we did to encode it, but instead of inserting in order of rows we insert in order
        # of column.
        n = len(self.key)
        no_rows = int(len(encodedmessage) / n) + 1 # Different formula for no. rows as encoded message will be 
        # the correct length already from the encode_message function
        key = self.key
        order = self.order
        C = np.full((no_rows,n), ' ', dtype=np.unicode_)
        # First row for the cipher in same way as we created the C matrix for encoding function
        for i in range(n):
            C[0,order[i]] = key[i]
        # Now fill in the data for the matrix in same way as for M in encoding function
        counter = 0
        next_spot = [1,0] 
        while counter < len(encodedmessage):
            C[next_spot[0],next_spot[1]] = encodedmessage[counter]
            if next_spot[0] == no_rows - 1:
                next_spot[0] = 1
                temp = next_spot[1]
                next_spot[1] = temp + 1
            else:
                temp = next_spot[0]
                next_spot[0] = temp + 1
            counter += 1
        # The difference here is that the encoded message will automatically fill the matrix
        # as it is made to in the encoding function so no need to add null values.
        # Matrix representing encoded data is complete. Now let's decode it.
        M = np.full((no_rows,n), ' ', dtype=np.unicode_)
        # First row is the original cipher
        for i in range(n):
            M[0,i] = key[i]
        # Now we rearrange the columns to decode the message
        for j in range(n):
            for i in range(1,no_rows):
                M[i,j] = C[i,order[j]]
        # That completes our matrix representing the decoded message
        # Let's unpack our M matrix into the decoded message and remove the trailing null values
        with_nulls = ''
        for i in range(1,no_rows):
            for j in range(n):
                with_nulls = with_nulls + M[i,j]
        while with_nulls[-1] == self.null:
            temp = with_nulls[:-1]
            with_nulls = temp
        encodedmessage = with_nulls
        return encodedmessage, M, C
    

    # Task 1.c
    def add(self, line):
        '''Adds a line to self.text having removed all new line characters
        Input: TranspositionCipher object and a string
        Result: Modified version of self.text'''
        backslash = '\\'
        corrected_line = ''
        counter = 0
        while counter < len(line):
            if line[counter] == backslash: # A newline character always has \n so need to skip over both characters
                counter += 2
            else: # Just add the character to the corrected version
                corrected_line = corrected_line + line[counter]
                counter += 1
        temp = self.text
        temp.append(corrected_line)
        self.text = temp
    
    # Task 1.c
    def encode(self):
        '''Converts all lines in self.text into their encoded version
        Input: TranspositionCipher object
        Result: Modifies self.text into list of encoded strings'''
        encoded_list = []
        text = self.text
        for line in text:
            encoded_version = self.encode_message(line)[0]
            encoded_list.append(encoded_version)
        self.text = encoded_list
    
    # Task 1.c
    def decode(self):
        '''Converts all lines in self.text into their decoded version
        Input: TranspositionCipher object
        Result: Modifies self.text into list of decoded strings'''
        decoded_list = []
        text = self.text
        for line in text:
            decoded_version = self.decode_message(line)[0]
            decoded_list.append(decoded_version)
        self.text = decoded_list

### Task 1.a Example

Copy the code for Task 1.a Example in the cell below and test to see if it has the required behaviour

In [5]:
cleanstring = clean('Hi! WhEre Will yoU be on 31/01/2024?')
print(cleanstring)

TC = TranspositionCipher()
print(TC)

fjord = TranspositionCipher(rawkey='Fjord', null='X')
print(fjord)

US = TranspositionCipher(rawkey='Unknown Soldier', null='Y')
print(US)

nephew = TranspositionCipher(rawkey="Dr. Anne-Marie's nephew", null='M')
print(nephew)

HiWhEreWillyoUbeon
Unknown { unkow : [3, 1, 0, 2, 4] : Z }
Fjord { fjord : [1, 2, 3, 4, 0] : X }
Unknown Soldier { unkowsldier : [9, 5, 3, 6, 10, 8, 4, 0, 2, 1, 7] : Y }
Dr. Anne-Marie's nephew { dranemisphw : [1, 8, 0, 6, 2, 5, 4, 9, 7, 3, 10] : M }


### Task 1.b Example

Copy the code for Task 1.b Example in the cell below and test to see if it has the required behaviour

In [6]:
message1 = 'I can resist everything except temptation.'

encoded1, M1encode, C1encode  = US.encode_message(message1)
print("Example 1, encoding:")
print(f"\noriginal message: '{message1:s}'")
print("\nmatrix form of original message:")
print(matrix_to_str(M1encode))
print("\nmatrix form of encoded message:")
print(matrix_to_str(C1encode))
print(f"\nencoded message: '{encoded1:s}'")


decoded1, M1decode, C1decode = US.decode_message(encoded1)
print("\nExample 1, decoding:")
print("\nmatrix form of encoded message:")
print(matrix_to_str(C1decode))
print("\nmatrix form of decoded message:")
print(matrix_to_str(M1decode))
print(f"\ndecoded message: '{decoded1:s}'")


message2 = 'We have been discovered. Return to base immediately.'
encoded2, M2e, C2e = TC.encode_message(message2)
decoded2, M2e, C2e = TC.decode_message(encoded2)
print(f"\nExamples 2: \nencoded: '{encoded2:s}'\ndecoded: '{decoded2:s}'")

Example 1, encoding:

original message: 'I can resist everything except temptation.'

matrix form of original message:
| u | n | k | o | w | s | l | d | i | e | r |
|---|---|---|---|---|---|---|---|---|---|---|
| I |   | c | a | n |   | r | e | s | i | s |
| t |   | e | v | e | r | y | t | h | i | n |
| g |   | e | x | c | e | p | t |   | t | e |
| m | p | t | a | t | i | o | n | . | Y | Y |

matrix form of encoded message:
| d | e | i | k | l | n | o | r | s | u | w |
|---|---|---|---|---|---|---|---|---|---|---|
| e | i | s | c | r |   | a | s |   | I | n |
| t | i | h | e | y |   | v | n | r | t | e |
| t | t |   | e | p |   | x | e | e | g | c |
| n | Y | . | t | o | p | a | Y | i | m | t |

encoded message: 'ettniitYsh .ceetrypo   pavxasneY reiItgmnect'

Example 1, decoding:

matrix form of encoded message:
| d | e | i | k | l | n | o | r | s | u | w |
|---|---|---|---|---|---|---|---|---|---|---|
| e | i | s | c | r |   | a | s |   | I | n |
| t | i | h | e | y |   | v | n | r | 

### Task 1.c Example

Copy the code for Task 1.c Example in the cell below and test to see if it has the required behaviour

In [7]:
legrand = TranspositionCipher("Legrand")
legrand.add("A good glass in the bishop's hostel in the devil's seat")
legrand.add("forty-one degrees and thirteen minutes")
legrand.add("northeast and by north")

print('original message:')
for t in legrand.text:
    print(f"'{t:s}'")

print('')
print('encoded:')
legrand.encode()
for t in legrand.text:
    print(f"'{t:s}'")

print('')
print('decoded:')
legrand.decode()
for t in legrand.text:
    print(f"'{t:s}'")

original message:
'A good glass in the bishop's hostel in the devil's seat'
'forty-one degrees and thirteen minutes'
'northeast and by north'

encoded:
'osepttia ibsle'Z l shid gathonesAgni   sd  'ehltoshos ve'
'yeariZordeuZoeetner sh sfne et-gntnZtd imZ'
'hnoZa tZotyZr  ZnsbhedrZtanZ'

decoded:
'A good glass in the bishop's hostel in the devil's seat'
'forty-one degrees and thirteen minutes'
'northeast and by north'


## Task 2
### Task 2 Code
Copy the skeletal code for Task 2 below and complete the implementation of Task 2.a-b as instructed:

In [8]:
class Play(object):
    def __init__(self):
        self.text = []
        return 

    # Task 2.a
    def read_from_file(self, filename):
        '''Goes through each line in the file and adds it to self.text, removing all trailing newline characters
        Input: Play object and a file
        Result: Modified self.text'''
        with open(filename,'r',encoding="utf-8") as file: # Automatically closes file once block of code finished
            for line in file.readlines():
                temp = self.text
                temp.append(line.strip('\n'))
                self.text = temp

    # Task 2.a 
    def write_to_file(self, filename):
        '''Writes each line in self.text to filename with each line in self.text getting a new line in the file.
        Input: Play object and a file
        Result: Modified file'''
        with open(filename,'w',encoding="utf-8") as file:
            for line in self.text:
                line_withnewline = line + '\n'
                file.write(line_withnewline)

    # Task 2.b
    def encode(self):
        '''Encodes the dialogue stored in self.text with keys corresponding to the speaker
        Input: Play object
        Result: self.text now has encoded dialogue'''
        # Let's use a while loop to go over the whole list and set up some important variables which will be
        # needed to distinguish if we are currently translating dialogue or looking at a speaker line.
        list_counter = 0
        text = self.text
        in_dialogue = False
        preceding_line_hasalpha = False
        current_key = ''
        while list_counter < len(text):
            line = text[list_counter]
            # Check if it has no alphabetical characters:
            has_alpha = False
            for character in line:
                counter = 0
                while has_alpha == False and counter < len(character):
                    if character[counter].isalpha() == True:
                        has_alpha = True
                    else:
                        counter += 1
            # If in dialogue either mark end of dialogue or encode line with the correct key
            if in_dialogue == True:
                if has_alpha == False: # Marks the end of the current dialogue
                    in_dialogue = False
                    preceding_line_hasalpha = False
                else: # If in dialogue and has alphabetical characters need to encode line and replace in the self.text
                    line_object = TranspositionCipher(rawkey=current_key,null='X')
                    encoded_line = line_object.encode_message(line)[0]
                    text[list_counter] = encoded_line
                    preceding_line_hasalpha = True
            else: # If not in dialogue check if the current line declares a new speaker
                # Let's check the criteria for a speaker line
                if preceding_line_hasalpha == False and has_alpha == True:
                    if line == line.upper() and line.strip()[-1] == '.': # Strip command can deal with extra whitespace at end of speaker line.
                        alpha_count = 0
                        for character in line:
                            if character.isalpha() == True:
                                alpha_count += 1
                        if alpha_count >= 3 and alpha_count <= 20: # Final criterion for being a speaker
                            keystring = ''
                            index_finalstop = len(line.strip()) - 1 # Need to stop at the final fullstop of the speaker name
                            for i in range(index_finalstop):
                                keystring = keystring + line[i]
                            current_key = keystring + ' speaks' # Sets up the key that will be used to encrypt the dialogue
                            in_dialogue = True # Declares start of a dialogue section
            list_counter += 1
        self.text = text
    
    # Task 2.b
    def decode(self):
        '''Decodes the dialogue stored in self.text assuming it has already been encoded
        Input: A Play object
        Result: self.text modified'''
        # This procedure should be extremely similar to the encoding task, as you need to distinguish between 
        # speaker and dialogue lines. In fact, the only difference in the entire function should be
        # the use of the encode_message vs decode_message function
        # Let's use a while loop to go over the whole list
        list_counter = 0
        text = self.text
        in_dialogue = False
        preceding_line_hasalpha = False
        current_key = ''
        while list_counter < len(text):
            line = text[list_counter]
            # Check if it has no alphabetical characters:
            has_alpha = False
            for character in line:
                counter = 0
                while has_alpha == False and counter < len(character):
                    if character[counter].isalpha() == True:
                        has_alpha = True
                    else:
                        counter += 1
            # If in dialogue either mark end of dialogue or decode line with the correct key
            if in_dialogue == True:
                if has_alpha == False: # Marks the end of the current dialogue
                    in_dialogue = False
                    preceding_line_hasalpha = False
                else: # If in dialogue and has alphabetical characters need to decode line and replace in self.text
                    line_object = TranspositionCipher(rawkey=current_key,null='X')
                    encoded_line = line_object.decode_message(line)[0]
                    text[list_counter] = encoded_line
                    preceding_line_hasalpha = True
            else: # If not in dialogue check if the current line declares a new speaker
                # Let's check the criteria for a speaker line
                if preceding_line_hasalpha == False and has_alpha == True:
                    if line == line.upper() and line.strip()[-1] == '.': # Strip command can deal with extra whitespace at end of speaker line.
                        alpha_count = 0
                        for character in line:
                            if character.isalpha() == True:
                                alpha_count += 1
                        if alpha_count >= 3 and alpha_count <= 20: # Final criterion for being a speaker
                            keystring = ''
                            index_finalstop = len(line.strip()) - 1 # Need to stop at the final fullstop of the speaker name
                            for i in range(index_finalstop):
                                keystring = keystring + line[i]
                            current_key = keystring + ' speaks' # Sets up the key that will be used to encrypt the dialogue
                            in_dialogue = True # Declares start of a dialogue section
            list_counter += 1
        self.text = text

### Task 2.a Example

Copy the code for Task 2.a Example in the cell below and test to see if it has the required behaviour

In [9]:
p = Play()
p.read_from_file(filename='task2example.txt')
print(p.text[0:5])
p.write_to_file(filename='task2example_copy.txt')
                 
q = Play()
q.read_from_file(filename='task2example_copy.txt')

q.text==p.text

['This is an example to demonstrate Task 2.', '', 'BOB.', 'Hi Alice, do you still have the coin that I gave you?', '']


True

### Task 2.b Example

Copy the code for Task 2.b Example in the cell below and test to see if it has the required behaviour

In [10]:
# load original play
p1 = Play()
p1.read_from_file('task2example.txt')

# create deepcopy, endcode and write to file
p2 = copy.deepcopy(p1)
p2.encode()
p2.write_to_file('task2example_encoded.txt')

# load encoded play, decode it and write to file
p3 = Play()
p3.read_from_file('task2example_encoded.txt')
p3.decode()
p3.write_to_file('task2example_decoded.txt')

# check that decoded play is the same as original play
p1.text==p3.text

True

## Additional Examples

* Any additional examples that you test your code with **can** be provided below. 

* Note that these are **NOT** a required part of the submission and **WILL NOT BE ASSESSED**. For this reason, if you prefer, you could also delete the examples below before submission and only use the space below for your own tests.

* Feel free to add more cells below as required. Reminder: The cells below will be ignored during assessment.

In [12]:
# Tests for 1a

cleanstring = clean('Hi! WhEre Will yoU be on 31/01/2024?')
print(cleanstring == 'HiWhEreWillyoUbeon')

TC = TranspositionCipher()
print(str(TC) == 'Unknown { unkow : [3, 1, 0, 2, 4] : Z }')

fjord = TranspositionCipher(rawkey='Fjord', null='X')
print(str(fjord) == 'Fjord { fjord : [1, 2, 3, 4, 0] : X }')

US = TranspositionCipher(rawkey='Unknown Soldier', null='Y')
print(str(US) == 'Unknown Soldier { unkowsldier : [9, 5, 3, 6, 10, 8, 4, 0, 2, 1, 7] : Y }')

nephew = TranspositionCipher(rawkey="Dr. Anne-Marie's nephew", null='M')
print(str(nephew) == '''Dr. Anne-Marie's nephew { dranemisphw : [1, 8, 0, 6, 2, 5, 4, 9, 7, 3, 10] : M }''')

True
True
True
True
True


In [13]:
# Tests for 1b:
# Correct version then mine
x = '''
Example 1, encoding:

original message: 'I can resist everything except temptation.'

matrix form of original message:
| u | n | k | o | w | s | l | d | i | e | r |
|---|---|---|---|---|---|---|---|---|---|---|
| I |   | c | a | n |   | r | e | s | i | s |
| t |   | e | v | e | r | y | t | h | i | n |
| g |   | e | x | c | e | p | t |   | t | e |
| m | p | t | a | t | i | o | n | . | Y | Y |

matrix form of encoded message:
| d | e | i | k | l | n | o | r | s | u | w |
|---|---|---|---|---|---|---|---|---|---|---|
| e | i | s | c | r |   | a | s |   | I | n |
| t | i | h | e | y |   | v | n | r | t | e |
| t | t |   | e | p |   | x | e | e | g | c |
| n | Y | . | t | o | p | a | Y | i | m | t |

encoded message: 'ettniitYsh .ceetrypo   pavxasneY reiItgmnect'

Example 1, decoding:

matrix form of encoded message:
| d | e | i | k | l | n | o | r | s | u | w |
|---|---|---|---|---|---|---|---|---|---|---|
| e | i | s | c | r |   | a | s |   | I | n |
| t | i | h | e | y |   | v | n | r | t | e |
| t | t |   | e | p |   | x | e | e | g | c |
| n | Y | . | t | o | p | a | Y | i | m | t |

matrix form of decoded message:
| u | n | k | o | w | s | l | d | i | e | r |
|---|---|---|---|---|---|---|---|---|---|---|
| I |   | c | a | n |   | r | e | s | i | s |
| t |   | e | v | e | r | y | t | h | i | n |
| g |   | e | x | c | e | p | t |   | t | e |
| m | p | t | a | t | i | o | n | . | Y | Y |

decoded message: 'I can resist everything except temptation.'

Examples 2: 
encoded: '   odttsmtZeencee ama.hbdv.uoeeeZWvesrRnbiiyaeie r  dlZ'
decoded: 'We have been discovered. Return to base immediately.'
'''

y = '''
Example 1, encoding:

original message: 'I can resist everything except temptation.'

matrix form of original message:
| u | n | k | o | w | s | l | d | i | e | r |
|---|---|---|---|---|---|---|---|---|---|---|
| I |   | c | a | n |   | r | e | s | i | s |
| t |   | e | v | e | r | y | t | h | i | n |
| g |   | e | x | c | e | p | t |   | t | e |
| m | p | t | a | t | i | o | n | . | Y | Y |

matrix form of encoded message:
| d | e | i | k | l | n | o | r | s | u | w |
|---|---|---|---|---|---|---|---|---|---|---|
| e | i | s | c | r |   | a | s |   | I | n |
| t | i | h | e | y |   | v | n | r | t | e |
| t | t |   | e | p |   | x | e | e | g | c |
| n | Y | . | t | o | p | a | Y | i | m | t |

encoded message: 'ettniitYsh .ceetrypo   pavxasneY reiItgmnect'

Example 1, decoding:

matrix form of encoded message:
| d | e | i | k | l | n | o | r | s | u | w |
|---|---|---|---|---|---|---|---|---|---|---|
| e | i | s | c | r |   | a | s |   | I | n |
| t | i | h | e | y |   | v | n | r | t | e |
| t | t |   | e | p |   | x | e | e | g | c |
| n | Y | . | t | o | p | a | Y | i | m | t |

matrix form of decoded message:
| u | n | k | o | w | s | l | d | i | e | r |
|---|---|---|---|---|---|---|---|---|---|---|
| I |   | c | a | n |   | r | e | s | i | s |
| t |   | e | v | e | r | y | t | h | i | n |
| g |   | e | x | c | e | p | t |   | t | e |
| m | p | t | a | t | i | o | n | . | Y | Y |

decoded message: 'I can resist everything except temptation.'

Examples 2: 
encoded: '   odttsmtZeencee ama.hbdv.uoeeeZWvesrRnbiiyaeie r  dlZ'
decoded: 'We have been discovered. Return to base immediately.'
'''

print(x==y)

True


In [14]:
# Tests for 1c:
# Correct version then mine
x = '''original message:
'A good glass in the bishop's hostel in the devil's seat'
'forty-one degrees and thirteen minutes'
'northeast and by north'

encoded:
'osepttia ibsle'Z l shid gathonesAgni   sd  'ehltoshos ve'
'yeariZordeuZoeetner sh sfne et-gntnZtd imZ'
'hnoZa tZotyZr  ZnsbhedrZtanZ'

decoded:
'A good glass in the bishop's hostel in the devil's seat'
'forty-one degrees and thirteen minutes'
'northeast and by north'
'''
y = '''original message:
'A good glass in the bishop's hostel in the devil's seat'
'forty-one degrees and thirteen minutes'
'northeast and by north'

encoded:
'osepttia ibsle'Z l shid gathonesAgni   sd  'ehltoshos ve'
'yeariZordeuZoeetner sh sfne et-gntnZtd imZ'
'hnoZa tZotyZr  ZnsbhedrZtanZ'

decoded:
'A good glass in the bishop's hostel in the devil's seat'
'forty-one degrees and thirteen minutes'
'northeast and by north'
'''
print(x==y)

True


In [15]:
# 2a

p = Play()
p.read_from_file(filename='task2example.txt')
print(p.text[0:5])
p.write_to_file(filename='task2example_copy.txt')
                 
q = Play()
q.read_from_file(filename='task2example_copy.txt')

q.text==p.text

['This is an example to demonstrate Task 2.', '', 'BOB.', 'Hi Alice, do you still have the coin that I gave you?', '']


True

In [16]:
# 2b Creating all the new files:

# load original play
p1 = Play()
p1.read_from_file('task2example.txt')

# create deepcopy, endcode and write to file
p2 = copy.deepcopy(p1)
p2.encode()
p2.write_to_file('task2example_encoded.txt')

# load encoded play, decode it and write to file
p3 = Play()
p3.read_from_file('task2example_encoded.txt')
p3.decode()
p3.write_to_file('task2example_decoded.txt')

# check that decoded play is the same as original play
print(p1.text==p3.text)

# load original play
p1 = Play()
p1.read_from_file('the_devils_disciple.txt')

# create deepcopy, endcode and write to file
p2 = copy.deepcopy(p1)
p2.encode()
p2.write_to_file('the_devils_disciple_encoded.txt')

# load encoded play, decode it and write to file
p3 = Play()
p3.read_from_file('the_devils_disciple_encoded.txt')
p3.decode()
p3.write_to_file('the_devils_disciple_decoded.txt')

# check that decoded play is the same as original play
print(p1.text==p3.text)

# load original play
p1 = Play()
p1.read_from_file('importance_of_being_earnest.txt')

# create deepcopy, endcode and write to file
p2 = copy.deepcopy(p1)
p2.encode()
p2.write_to_file('importance_of_being_earnest_encoded.txt')

# load encoded play, decode it and write to file
p3 = Play()
p3.read_from_file('importance_of_being_earnest_encoded.txt')
p3.decode()
p3.write_to_file('importance_of_being_earnest_decoded.txt')

# check that decoded play is the same as original play
print(p1.text==p3.text)

True
True
True


In [17]:
# Tests to check if all the self.texts are the same:

# For task2example
task1enc = Play()
task1enc.read_from_file('task2example_encoded.txt')
task2enc = Play()
task2enc.read_from_file('task2example_encoded-Copy1.txt')

task1dec = Play()
task1dec.read_from_file('task2example_decoded.txt')
task2dec = Play()
task2dec.read_from_file('task2example_decoded-Copy1.txt')

print(task1enc.text == task2enc.text)
print(task1dec.text == task2dec.text)

# For the devils disciple

dev1enc = Play()
dev1enc.read_from_file('the_devils_disciple_encoded.txt')
dev2enc = Play()
dev2enc.read_from_file('the_devils_disciple_encoded-Copy1.txt')

dev1dec = Play()
dev1dec.read_from_file('the_devils_disciple_decoded.txt')
dev2dec = Play()
dev2dec.read_from_file('the_devils_disciple_decoded-Copy1.txt')

print(dev1enc.text == dev2enc.text)
print(dev1dec.text == dev2dec.text)

# For the importance of being earnest

im1enc = Play()
im1enc.read_from_file('importance_of_being_earnest_encoded.txt')
im2enc = Play()
im2enc.read_from_file('importance_of_being_earnest_encoded-Copy1.txt')

im1dec = Play()
im1dec.read_from_file('importance_of_being_earnest_decoded.txt')
im2dec = Play()
im2dec.read_from_file('importance_of_being_earnest_decoded-Copy1.txt')

print(im1enc.text == im2enc.text)
print(im1dec.text == im2dec.text)

True
True
True
True
True
True
